In [ ]:
# Cell 1: Import & Konfigurasi
import pandas as pd
import ast
import gc
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Konfigurasi File
INPUT_FILE = 'Data/Data_Siap_Mining.csv' # Sesuaikan path jika perlu
OUTPUT_FILE = 'Final_Skripsi_Rules.csv'

# Konfigurasi Algoritma
MIN_SUPPORT_COUNT = 20
MIN_CONFIDENCE = 0.6

print("✅ Library loaded & Config set.")

# Analisis Anomali Network Menggunakan FP-Growth dan Association Rules

## Deskripsi Proyek
Notebook ini menganalisis log jaringan untuk mendeteksi anomali menggunakan algoritma **FP-Growth** (Frequent Pattern Growth) dan **Association Rules**. Tujuan utama adalah menemukan pola hubungan antara kondisi jaringan dan diagnosis anomali.

## Struktur Analisis
1. **Encoding Data**: Mengonversi data log menjadi format transaksi yang dapat dianalisis
2. **FP-Growth Algorithm**: Menemukan pola frekuen dengan threshold minimum support
3. **Association Rules**: Menghasilkan aturan asosiasi untuk prediksi diagnosis
4. **Filtering & Visualization**: Menyaring aturan yang relevan dan menampilkan hasil akhir

## Tahap 1: Import Library dan Persiapan Data

Pada tahap ini, kami mengimpor library yang diperlukan untuk analisis frequent pattern mining dan memuat dataset yang sudah dibersihkan dari tahap preprocessing.

## Tahap 2: Encoding Data dan Perhitungan FP-Growth

### Proses Encoding
- **TransactionEncoder**: Mengkonversi data log dari format text menjadi format transaksi (one-hot encoded)
- Setiap item dalam 'items' column diubah menjadi kolom boolean terpisah
- Output: DataFrame `df_encoded` dengan nilai True/False untuk setiap atribut

### Perhitungan Support
- **Minimum Support**: Threshold minimum untuk pola yang dianggap "sering" terjadi
- **MIN_SUPPORT_COUNT = 20**: Pola harus muncul minimal 20 kali dalam dataset
- **min_support_pct**: Persentase support dihitung dari total jumlah transaksi

### Algoritma FP-Growth
- **fpgrowth**: Algoritma efisien untuk mining pola frekuen
- Lebih cepat dari Apriori karena menggunakan FP-tree structure
- Output: `frequent_itemsets` berisi semua pola yang memenuhi threshold

### Association Rules
- Menghasilkan aturan IF-THEN dari pola frekuen
- **min_threshold = 0.6**: Hanya aturan dengan confidence ≥ 60% yang dipertahankan
- Metrik yang dihitung: support, confidence, lift, dsb.

## Tahap 3: Penyaringan dan Tampilan Hasil Akhir

### Konsep Filtering
Dari semua association rules yang dihasilkan, kita hanya ingin menyimpan rules yang:
- **Consequent (Kesimpulan)**: Harus berupa diagnosis/label anomali
  - NORMAL: Kondisi jaringan normal
  - UPSTREAM_FAILURE: Kegagalan uplink
  - LINK_FAILURE: Kegagalan link
  - DDOS_ATTACK: Serangan DDoS
  - BROADCAST_STORM: Broadcast storm

Contoh Rule yang BENAR (yang kita inginkan):
- IF `ether1=down AND queue=high` THEN `LINK_FAILURE` ✓

Contoh Rule yang SALAH (yang kita buang):
- IF `LINK_FAILURE` THEN `ether1=down` ✗

### Metrik Evaluasi
- **Support**: Seberapa sering pola ini muncul dalam dataset (0-1)
- **Confidence**: Probabilitas consequent terjadi jika antecedent terjadi (0-1)
- **Lift**: Seberapa kuat hubungan antara antecedent dan consequent (>1 = hubungan positif)

## Kesimpulan dan Interpretasi

### Hasil Analisis
Notebook ini menghasilkan association rules yang menunjukkan hubungan antara kondisi jaringan dan tipe anomali. Setiap rule memiliki:
- **Antecedents (IF)**: Kondisi atau kombinasi kondisi jaringan yang diamati
- **Consequents (THEN)**: Diagnosis/prediksi tipe anomali yang kemungkinan terjadi
- **Confidence**: Tingkat akurasi prediksi (semakin tinggi semakin baik)
- **Support**: Seberapa sering pola ini muncul dalam data (frekuensi)
- **Lift**: Seberapa kuat hubungan antara kondisi dan diagnosis

### Penggunaan Praktis
Hasil rules ini dapat digunakan untuk:
1. **Early Warning System**: Mendeteksi anomali sebelum terjadi dengan mengecek kondisi antecedent
2. **Root Cause Analysis**: Memahami faktor-faktor apa yang menyebabkan setiap tipe anomali
3. **Network Optimization**: Meningkatkan monitoring dan maintenance protokol berdasarkan insights
4. **Decision Support**: Membantu network administrator membuat keputusan yang lebih cepat dan tepat

### File Output
- **Final_Skripsi_Rules.csv**: Berisi top 10 rules dalam format CSV yang dapat digunakan untuk proses selanjutnya

In [ ]:
# ===== STEP 1: LOAD & PREPARE TRANSACTIONS =====
import pandas as pd
import ast
import gc
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load data yang sudah bersih
df_cleaned = pd.read_csv('Data/Data_Siap_Mining.csv')
print(f"📊 Dataset dimuat: {len(df_cleaned)} baris, {df_cleaned.shape[1]} kolom")

# Deteksi nama kolom label (antisipasi beda nama)
label_col = 'Label' if 'Label' in df_cleaned.columns else 'diagnosis'
print(f"ℹ️ Menggunakan kolom label: {label_col}")

# GABUNGKAN Gejala (Items) + Label (Diagnosis) -> INI KUNCINYA!
transactions = []
for _, row in df_cleaned.iterrows():
    # Ubah string log kembali jadi list (GUNAKAN ast.literal_eval - LEBIH AMAN)
    items = ast.literal_eval(row['items']) if isinstance(row['items'], str) else row['items']
    # MASUKKAN Label Diagnosis ke dalam list agar ikut dipelajari
    items.append(row[label_col])
    transactions.append(items)

print(f"✓ Transaksi siap. Contoh: {transactions[0]}")

# Hapus DF awal untuk hemat memori
del df_cleaned
gc.collect()

# ===== STEP 2: ENCODING =====
print("\n⏳ Sedang melakukan Encoding...")
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# Hapus list transaksi segera
del transactions
gc.collect()

df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Hapus array numpy segera
del te_ary
gc.collect()

print(f"✓ Encoding Selesai. Shape Matriks: {df_encoded.shape}")

# ===== STEP 3: FP-GROWTH MINING =====
MIN_SUPPORT_COUNT = 20
MIN_CONFIDENCE = 0.6
min_support_pct = MIN_SUPPORT_COUNT / len(df_encoded)
print(f"\n🔍 Mining dengan Min Support: {min_support_pct:.5f} (Min {MIN_SUPPORT_COUNT} kejadian)")

# Jalankan FP-Growth
frequent_itemsets = fpgrowth(df_encoded, min_support=min_support_pct, use_colnames=True)

# Hapus matriks besar segera (Hemat RAM)
del df_encoded
gc.collect()

print(f"✓ Mining Selesai. Ditemukan {len(frequent_itemsets)} pola itemset.")

In [ ]:
# ===== STEP 4: GENERATE RULES =====
print(f"\n⚙️ Generating Rules (Min Conf: {MIN_CONFIDENCE})...")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)

# Bersihkan memori
del frequent_itemsets
gc.collect()

# ===== STEP 5: FILTERING DIAGNOSIS =====
# Hanya simpan rule yang ujungnya (Consequents) adalah Diagnosis
TARGET_LABELS = ['NORMAL', 'UPSTREAM_FAILURE', 'LINK_FAILURE', 'DDOS_ATTACK', 'BROADCAST_STORM']

def is_diagnosis_rule(consequents):
    return any(label in consequents for label in TARGET_LABELS)

if not rules.empty:
    final_rules = rules[rules['consequents'].apply(is_diagnosis_rule)].copy()
else:
    final_rules = pd.DataFrame()

# ===== STEP 6: FORMATTING & SAVING =====
if not final_rules.empty:
    # Sort dari yang paling akurat
    final_rules = final_rules.sort_values(['confidence', 'support'], ascending=[False, False])

    # Ubah format frozenset jadi list biasa (Wajib untuk Dashboard)
    final_rules['antecedents'] = final_rules['antecedents'].apply(lambda x: list(x))
    final_rules['consequents'] = final_rules['consequents'].apply(lambda x: list(x))

    # Tampilkan Top 10 (Hanya untuk dilihat mata)
    print(f"\n{'='*50}")
    print(f"TOP 10 PREVIEW (Dari total {len(final_rules)} rules)")
    print(f"{'='*50}")
    for idx, row in final_rules.head(10).iterrows():
        print(f"IF {row['antecedents']} THEN {row['consequents']} (Conf: {row['confidence']:.2f})")

    # SIMPAN SEMUA RULES KE CSV (PENTING: Jangan pakai .head() disini!)
    output_path = 'Final_Skripsi_Rules.csv'
    final_rules.to_csv(output_path, index=False)
    
    print(f"\n{'='*50}")
    print(f"✅ SUKSES! Total {len(final_rules)} Rules valid disimpan.")
    print(f"💾 File: {output_path}")
    print(f"{'='*50}")

else:
    print("❌ PERINGATAN: Tidak ada rule diagnosis yang ditemukan! Cek apakah label sudah masuk ke transaksi.")